# 处理搜狐数据，去掉article content中带有html标签

In [ ]:
import sys
sys.path.append('../utils')
import sql_util as sql
from bs4 import BeautifulSoup
import re
import time
import threading

In [ ]:
class Thread(threading.Thread):
    def __init__(self, num, lst):
        self._num = num
        self._lst = lst
        super().__init__()

    def run(self):
        print("[{}]--process start, process {} thread, num is {}......".format(
            time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()), self._num, len(self._lst)))
        # 循环处理数据，去掉每一条数据中的article_content html标签
        # 正则表达式过滤标签<>...</> or <.../>
        # for element in self._lst:
        #     file_util.write_file(os.path.join(self._path, element[0] + '.txt'),
        #                          element[1])
        count = 0
        for element in self._lst:
            temp = re.compile(r'<[^>]*>', re.S)
            temp = temp.sub('', element.get('article_content'))
            temp = str.replace(temp, '\n', '').strip()
            finally_result = sql.execute("update sj_sohu_no_html set article_content = %s where id = %s", (temp, element.get('id')))
            if finally_result == 0:
                print('failed...')
            count += 1
        print("[{}]--process end......".format(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())))

In [ ]:
def start(step=5000):
    result_data = sql.queryall('select * from sj_sohu_no_html')
    temp_list = [result_data[i:i + step] for i in range(0, len(result_data), step)]
    thr_list = [Thread(i, temp_list[i]) for i in range(len(temp_list))]
    [thr.start() for thr in thr_list]
    [thr.join() for thr in thr_list]

In [ ]:
start()

In [ ]:
# 查询所有数据
result_data = sql.queryall('select * from sj_sohu_no_html limit 1')
# print(result_data[0].get('article_content').strip())
print(str.replace(result_data[0].get('article_content'), '\n', '').strip())

In [39]:
result_data = sql.queryall('select * from sj_sohu_no_html')
print('数据总数: {}'.format(len(result_data)))

数据总数: 144523
